In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx

from urllib.parse import urljoin

from will_it_saturate.server import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.core import FilesystemCreator, Epoch, Result, BenchmarkFile
from will_it_saturate.client import HttpxClient, AioHttpClient, WrkClient
from will_it_saturate.persistence import SqliteRepository

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7

epoch = Epoch(file_size=file_size, duration=duration)

<IPython.core.display.Javascript object>

In [ ]:
control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict_without_file_creator())
print(r.status_code, len(r.json()))

200 6


<IPython.core.display.Javascript object>

In [ ]:
r.json()

{'file_size': 10000000,
 'duration': 3,
 'bandwidth': 125000000,
 'files': [{'number': 0,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '4a45eb0c5580122823126fc30477ce08',
   'creator': {}},
  {'number': 1,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '22b964829c3cd80f9627861eee97ad6f',
   'creator': {}},
  {'number': 2,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '2313682c8f900d9011cf3557356c0b75',
   'creator': {}},
  {'number': 3,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '9f09900cb8d58d1e470e087682f385b2',
   'creator': {}},
  {'number': 4,
   'base_path': '10000000_3_125000000',
   'si

<IPython.core.display.Javascript object>

In [ ]:
for bfile in r.json()["files"]:
    kwargs = {k: v for k, v in bfile.items() if k != "creator"}
    epoch.files.append(BenchmarkFile(**kwargs))

<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()

<IPython.core.display.Javascript object>

In [ ]:
server.dict()

{'name': 'fastAPI/uvicorn', 'host': 'localhost', 'port': 8000}

<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
server = FastAPIUvicornServer(**r.json())
print(r.status_code, r.json())

200 {'name': 'fastAPI/uvicorn', 'host': 'localhost', 'port': 5001}


<IPython.core.display.Javascript object>

In [ ]:
server

FastAPIUvicornServer(name='fastAPI/uvicorn', host='localhost', port=5001)

<IPython.core.display.Javascript object>

In [ ]:
for efile in epoch.files:
    efile.port = 5001

<IPython.core.display.Javascript object>

In [ ]:
client = HttpxClient(name="httpx")


result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

In [ ]:
%%time
result.elapsed = client.measure(epoch)

measure
CPU times: user 8.64 ms, sys: 12.3 ms, total: 20.9 ms
Wall time: 9.41 s


<IPython.core.display.Javascript object>

In [ ]:
result

Result(server='fastAPI/uvicorn', client='httpx', file_size=10000000, elapsed=6.730053406, complete_size=375000000)

<IPython.core.display.Javascript object>

In [ ]:
result.bytes_per_second

55720211.6205614

<IPython.core.display.Javascript object>

# Local Benchmark

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

epoch.create_files()

result = BenchmarkResult(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

result.elapsed = client.measure(epoch)

<IPython.core.display.Javascript object>